In [1]:
import pandas as pd
import numpy as np
import itertools
import copy

from collections import OrderedDict
from operator import itemgetter

# Programming Assignment: Рекомендательные системы

#### https://www.coursera.org/learn/data-analysis-applications/programming/OSjrS/riekomiendatiel-nyie-sistiemy

### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать __recall@k__ и __precision@k__.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

#### Входные данные

Вам дается две выборки с пользовательскими сессиями - __id__-шниками просмотренных и __id__-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: __id__ просмотренных товаров через , затем идёт ; после чего следуют __id__ купленных товаров (если такие имеются), разделённые запятой. Например, __1,2,3,4__; или __1,2,3,4;5,6__.

Гарантируется, что среди __id__ купленных товаров все различные.

#### Важно:

* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* комендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и __k__ в __recall@k / precision@k__.

#### Задание

1. На обучении постройте частоты появления __id__ в просмотренных и в купленных (__id__ может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    * сортировка просмотренных __id__ по популярности (частота появления в просмотренных),
    * сортировка просмотренных __id__ по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел __AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5__ на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

#### Дополнительные вопросы

1. Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров __recall@k__ меняется. Подумайте, как оценить минимальное и максимальное значение __recall@k__ в зависимости от правила сортировки.
2. Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [2]:
def process_line(line, dic):
    items = [int(x) for x in line.split(',') if x != '']
    
    # assign value to the row
    for item in items:
        if item not in dic:
            dic[item] = 0.0
        dic[item] += 1.0

def process_data(file_name):
    observed_data = dict()
    bought_data = dict()
    
    with open(file_name) as file_obj:
        for line in file_obj:
            (watch, buy) = zip(line.replace('\n', '').split(';'))
            
            process_line(watch[0], observed_data)
            process_line(buy[0], bought_data)

    return (observed_data, bought_data)

In [3]:
#(train_observed, train_bought) = process_data('test.txt')
(train_observed, train_bought) = process_data('coursera_sessions_train.txt')

In [4]:
len(train_observed)
#train_observed

77064

In [5]:
len(train_bought)

4479

In [24]:
STEP_WEIGHT = 0.01
def get_recommendations(popularity_dict, observed, k):
    watch = list()
    for obs in observed:
        if obs not in watch:
            watch.append(obs)
    
    recommended = dict()
    for obs in watch:
        if obs in popularity_dict:
            recommended[obs] = popularity_dict[obs]
        else:
            recommended[obs] = 0.0
    # iterate over reversed observed items to add weight for first seen items
    for i, item_id in enumerate(watch[::-1]):
        if item_id in recommended:
            recommended[item_id] += (i * STEP_WEIGHT)
    
    recommended = sorted(recommended.items(), key=itemgetter(1), reverse=True)
    items_to_recommend = min(len(observed), k)
    ids_to_recommend = [t[0] for t in recommended[:items_to_recommend]]
    return (ids_to_recommend, recommended)

In [41]:
recommended, details = get_recommendations(train_observed, [11, 9, 0, 21], 3)

print recommended
print details
print train_observed

[11, 9, 0]
[(11, 2.03), (9, 2.02), (0, 1.01), (21, 1.0)]
{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 9: 2.0, 10: 1.0, 11: 2.0, 16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0, 21: 1.0}


In [27]:
def quality(recommended, bought, k, metrics):
    if metrics not in ['precision', 'recall']:
        raise ValueError('%s is wrong, use "precision" or "recall"' % metrics) 
    
    correct_count = 0
    precision = 0.0
    for step,rec in enumerate(recommended, start=1):
        corr = 1 if rec in bought else 0
        correct_count += corr
        
    divider = k if metrics == 'precision' else len(bought)
    quality = correct_count / float(divider)
    
    return quality

In [8]:
print quality([9, 12, 13, 10], [9, 10, 2, 3], 4, 'precision')
print quality([9, 10, 12, 13], [9, 10, 2, 3], 4, 'precision')

print quality([9, 12, 13, 10], [9, 10, 2, 3], 4, 'recall')
print quality([9, 10, 12, 13], [9, 10, 2, 3], 4, 'recall')

0.5
0.5
0.5
0.5


In [25]:
def collect_metrics(file_name, popular_dict, debug=False):
    av_precision_1 = list()
    av_precision_5 = list()
    
    av_recall_1 = list()
    av_recall_5 = list()
    
    count = 0
    
    with open(file_name) as file_obj:
        for line in file_obj:
            (watch, buy) = zip(line.replace('\n', '').split(';'))
            
            watched_items = [int(x) for x in watch[0].split(',') if x != '']
            bought_items = [int(x) for x in buy[0].split(',') if x != '']
            
            if len(bought_items) == 0:
                continue
            
            recommend_watch_1 = get_recommendations(popular_dict, watched_items, 1)[0]
            recommend_watch_5 = get_recommendations(popular_dict, watched_items, 5)[0]
            
            precision_1 = quality(recommend_watch_1, bought_items, 1, 'precision')
            precision_5 = quality(recommend_watch_5, bought_items, 5, 'precision')
            
            recall_1 = quality(recommend_watch_1, bought_items, 1, 'recall')
            recall_5 = quality(recommend_watch_5, bought_items, 5, 'recall')
            
            av_precision_1.append(precision_1)
            av_precision_5.append(precision_5)
            
            av_recall_1.append(recall_1)
            av_recall_5.append(recall_5)
            
            count += 1
            
            
            if debug == True:
                print '----------------'
                print 'watched: %s' % watched_items
                print 'recommend_watch_1: %s' % recommend_watch_1
                print 'recommend_watch_5: %s' % recommend_watch_5
                print 'bought_items: %s' % bought_items
                print 'recall_1: %f' % recall_1
                print 'precision_1: %f' % precision_1
                print 'recall_5: %f' % recall_5
                print 'precision_5: %f' % precision_5
                print '----------------'
            
                if count >= 2:
                    break
            
    return (av_precision_1, av_precision_5, av_recall_1, av_recall_5)

In [26]:
#(av_precision_1, av_precision_5, av_recall_1, av_recall_5) = collect_metrics('coursera_sessions_train.txt', train_observed, debug=False)
#(av_precision_1, av_precision_5, av_recall_1, av_recall_5) = collect_metrics('coursera_sessions_test.txt', train_observed, debug=False)
#(av_precision_1, av_precision_5, av_recall_1, av_recall_5) = collect_metrics('coursera_sessions_train.txt', train_bought, debug=False)
(av_precision_1, av_precision_5, av_recall_1, av_recall_5) = collect_metrics('coursera_sessions_test.txt', train_bought, debug=False)

print 'av_recall_1: %.2f' % np.mean(av_recall_1)
print 'av_precision_1: %.2f' % np.mean(av_precision_1)

print 'av_recall_5: %.2f' % np.mean(av_recall_5)
print 'av_precision_5: %.2f' % np.mean(av_precision_5)

av_recall_1: 0.46
av_precision_1: 0.53
av_recall_5: 0.82
av_precision_5: 0.21
